In [6]:
from pytube import YouTube
import os
import pydub
from pydub.utils import which
from pydub import AudioSegment
import openai

In [7]:
AudioSegment.converter = which("ffmpeg")


In [41]:
def download_audio():
    url = input("Introduce the YouTube URL:")
    try:
        if "https://www.youtube.com/" in url:
            try:
                yt = YouTube( 
                str(url)) 
            except Exception as ee:
                print(ee)

            # extract only audio 
            audio = yt.streams.filter(only_audio=True).first() 
        
            destination = '.'
        
            # download the file 
            out_file = audio.download(output_path=destination) 
        
            # save the file 
            base, ext = os.path.splitext(out_file) 
            new_file = (base + '.mp3').replace(" ", "_")
            os.rename(out_file, new_file) 
        
            # result of success 
            print(yt.title + " has been successfully downloaded.")
            return new_file
        else:
            print("This is not a valid URL!")
    except Exception as ee:
        print(ee)

In [44]:
file_name = download_audio()

Build a Chatbot in Python| ChatGPT API Complete Tutorial for Beginners has been successfully downloaded.


In [17]:
def check_audio_size(file_location):
    file_stats = os.stat(file_location)
    file_size = file_stats.st_size / (1024*1024) # In MB
    print(f"Audio file: {file_size:.2f} MB")
    return file_size

In [46]:
def split_audio(file_name):
    try:
        mp3_audio = pydub.AudioSegment.from_file(file_name, "mp3")
    except Exception as ee:
        mp3_audio = pydub.AudioSegment.from_file(file_name, "mp4")

    chunk_size = 10 * 1024 * 1024  # 10 MB in bytes
    num_chunks = len(mp3_audio) // chunk_size + 1
    
    chunk_names = []
    for counter in range(num_chunks):
        start = counter * chunk_size
        end = min(start + chunk_size, len(mp3_audio))
        chunk = mp3_audio[start:end]
        chunk_name = "output_" + str(counter + 1) + ".mp3"
        chunk.export(chunk_name, format="mp3")
        chunk_names.append(chunk_name)
    return chunk_names


In [49]:
file_size = check_audio_size(file_name)
if file_size > 20:
    chunk_names = split_audio(file_name)
else:
    chunk_names = [file_name]
    print('Correct size!')

Audio file: 2.85 MB
Correct size!


In [35]:
for counter in range(num_chunks):
    start = counter * chunk_size
    end = min(start + chunk_size, len(mp3_audio))
    chunk = mp3_audio[start:end]
    chunk.export("output_" + str(counter + 1) + ".mp3", format="mp3")


In [9]:
client = openai.OpenAI()

In [11]:
transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=open(new_file, "rb"),
    language="en"
)

transcript.text

"Hello everyone. My name is Nachiketa and welcome back to another video. In this video, I'm going to show you how you can use ChatGPT to create your own personal chatbot for any of your application. You might have already heard a lot about ChatGPT. Just to summarize, ChatGPT is basically an AI-powered chatbot which uses something called as GPT or basically a generative pre-trained transformer, which is basically a language model which allows you to talk to it like a human does. But if you're building an application, let's say you're building a website, you're building a mobile phone app and you want to build a chatbot into it, you don't want the end-user to directly go into the ChatGPT website and enter, let's say, a prompt and get the answer himself. You want them to be able to do it from your application itself. Now, since you don't want the end-user to go to ChatGPT himself, what ChatGPT has done is they have created a public API using which, from your application, you can take inpu

In [14]:
doc_content = transcript.text
prompt = f"""   
    It is necessary to generate a detailed summary about the following text 
    that is extracted from a youtube video:
    '''
        {doc_content}
    '''
    Please return the summary in a json format containing the following fields:
    '''
        "Video topic": ...
        "Summary": ...
        "Keywords": ...
    '''
"""

#response = openai.ChatCompletion.create(
#    model = 'gpt-3.5-turbo',
#    messages = [
#        {
#            "role": "user",
#            "content": prompt
#        }
#    ]
#)
#print(response['choices'][0]['messages']['content'])

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt 
        }
    ],
    model="gpt-3.5-turbo-1106",
    response_format={"type": "json_object"},
    temperature=1.9,
)
print(response.model_dump()['choices'][0]['message']['content'])

{
  "Video topic": "Using ChatGPT for building personal chatbots",
  "Summary": "The video provides a tutorial on how to use the ChatGPT public API to build a personalized chatbot for websites or mobile apps. The speaker explains the process of creating personalized responses based on user data and API usage to control costs. The tutorial includes steps such as creating an OpenAI API key, using the library, setting up messages for communication, and controlling costs by managing the number of tokens used.",
  "Keywords": "ChatGPT, chatbot, public API, OpenAI, AI applications, user data, library, API usage control, tokens"
}


In [36]:

def split_mp3_file(input_file):
    chunk_size = 10 * 1024 * 1024 
    try:
        mp3_audio = pydub.AudioSegment.from_file(input_file, "mp3")
    except Exception as ee:
        mp3_audio = pydub.AudioSegment.from_file(input_file, "mp4")
    num_chunks = len(input_file) // chunk_size + 1

    output_folder = "chunks"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i in range(num_chunks):
        start = i * chunk_size
        end = min(start + chunk_size, len(input_file))
        chunk = input_file[start:end]

        chunk_filename = f"output_{i + 1}.mp3"
        chunk_path = os.path.join(output_folder, chunk_filename)
        chunk.export(chunk_path, format="mp3")


split_mp3_file(mp3_audio)


AttributeError: 'AudioSegment' object has no attribute 'read'